# Day 3

In Part 1, we find the coordinates within each fabric swatch that overlap with at least one other and return those and produce a count.  In Part 2, we find the single fabric ID that has no coordinates in common with any others.

A somewhat expected interpretation on this problem is to represent it as a grid and then ostensibly you can mark and sweep the grid as you work through each fabric.  Likely you'd need to pre-process the list of fabrics and the coordinates to know how big of a grid to build (which could be quite big) though since everythign is coordinate based you'd have O(1) lookups into any position once you'd initialized the grid.  I went a different way on this, though, and viewed the problem through the lens of set membership.  Basically, each fabric I viewed as comprising a set of coordinates.  I then used set membership functions to compare that set to the sets for other fabrics to ultimately produce a final overlapping set. I don't know that performance or space complexity is much different since in both cases you'd need a structure that holds or represents the coordinates.  Since I hold both the intersected coordinates (the overlaps) and all of them (the union), I'm not really saving space though since every lookup I perform is through the set, I can get O(1) on testing memberships.  A possible optimization is to consider a way to assess the overlap of two fabrics without the need to hold the full set of coordinates thereafter, such as a bounding box overlap algorithm.  

In Part 2, the requirement was to find the non-overlapping fabric.  My first function to find the overlapping coords didn't have an easy modification within its logic to add that in.  However, I was able to build on top of it by first finding the overlapping coords, then retesting each fabric to find the fabric whose coordinates were disjoint with the overlapping set, the idea being that the target fabric contributed nothing to the overlapping set so the intersection would be empty.  

In [ ]:
from utils import read_input

In [ ]:
def parse_input(line):
    
    # Parts will contain ['#X', '@', 'Y,Z:', 'AxB'] 
    # where X, Y, Z, A, and B are integers with X as ID, Y and Z as left and top offset, and A and B as box dimensions
    parts = line.split()    
    
    # Split again on # and take the second portion as the ID.  Leave as a string.
    fabric_id = parts[0].split('#')[1]
    
    # Split the offset string on comma.  The first part can be converted to int as-is, for second we must deal with 
    # the : so we'll just chomp it off using the array subscript
    offset = parts[2].split(',')    
    upper_left_offset = (int(offset[0]), int(offset[1][:-1]))
                        
    fabric_size_string = parts[3].split('x')
    fabric_size = (int(fabric_size_string[0]), int(fabric_size_string[1]))
    
    return (fabric_id, upper_left_offset, fabric_size)
    
    

In [ ]:
def all_coords_for_fabric(upper_left_offset, dimensions):
    
    left_offset, top_offset = upper_left_offset
    width, height = dimensions
  
    coords = set([])
    
    for x in range(left_offset, left_offset + width):
        for y in range(top_offset, top_offset + height):
            coords.add((x,y))
            
    return coords
    

In [ ]:
# For the given set of fabrics where each fabric is a tuple (<ID>, (<UpperLeftOffset>), (<Dimensions>)), returns
# the set of coordinates across all fabrics that overlap (i.e. where the coordinate is used by more than 1 fabric)
def find_overlap(fabrics): 
    
    unioned = set([])
    overlapped = set([])    
    
    for fabric in fabrics:
        
        current = all_coords_for_fabric(fabric[1], fabric[2])
        
        new_overlapping = unioned.intersection(current)
        
        unioned = unioned.union(current)

        overlapped = overlapped.union(new_overlapping)
        
    return overlapped

# For the given set of fabrics and the set of coordinates that overlap across those fabrics, returns
# the first ID of a fabric that is disjoint from the overlap (i.e. that has no coordinates that overlap
# any other fabric).
def find_non_overlapped(fabrics, overlapped):    
           
    for fabric in fabrics:
        
        fabric_coords = all_coords_for_fabric(fabric[1], fabric[2])
        
        if not fabric_coords.intersection(overlapped):
            return fabric[0]
        
    return None
    

In [ ]:
def solve():
    fabrics = read_input('Input\day3.txt', parse_input)
    
    overlapped = find_overlap(fabrics)
    
    print '{} overlapping units'.format(len(overlapped))
    
    disjoint_fabric_id = find_non_overlapped(fabrics, overlapped)
    
    print 'Fabric ID {} has nothing in common with the overlapped coords'.format(disjoint_fabric_id)
    
    print 'Done'
    

In [ ]:
solve()